In [32]:
!conda install -c rapidsai -c conda-forge cudf-polars-cu12

2 channel Terms of Service accepted
Retrieving notices: done
Channels:
 - rapidsai
 - conda-forge
 - defaults
Platform: osx-arm64
Solving environment: failed

PackagesNotFoundError: The following packages are not available from current channels:

  - cudf-polars-cu12

Current channels:

  - https://conda.anaconda.org/rapidsai
  - https://conda.anaconda.org/conda-forge
  - defaults

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




In [1]:
import polars as pl
import pandas as pd

In [2]:
import sklearn
import pandas as pd
df = sklearn.datasets.fetch_20newsgroups(subset='train')

In [2]:
df = pd.read_csv('Articles.csv', encoding='latin-1')
df.head()

,Article,Date,Heading,NewsType
0,KARACHI: The Sindh government has decided to b...,1/1/2015,sindh govt decides to cut public transport far...,business
1,HONG KONG: Asian markets started 2015 on an up...,1/2/2015,asia stocks up in new year trad,business
2,HONG KONG: Hong Kong shares opened 0.66 perce...,1/5/2015,hong kong stocks open 0.66 percent lower,business
3,HONG KONG: Asian markets tumbled Tuesday follo...,1/6/2015,asian stocks sink euro near nine year,business
4,NEW YORK: US oil prices Monday slipped below $...,1/6/2015,us oil prices slip below 50 a barr,business


In [3]:
df.shape

(2692, 4)

In [8]:
df_pandas = df.copy()
df_pandas['article_id'] = range(1, len(df_pandas) + 1)
df_pandas['publication'] = 'Boston Globe'
df_pandas = df_pandas.rename(columns={'Date':'date'})
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

# Download required NLTK data
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('punkt', quiet=True)

# Initialize lemmatizer and stop words
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    """Remove stop words and lemmatize text"""
    if pd.isna(text):
        return ""
    
    # Convert to lowercase and remove non-alphabetic characters
    text = re.sub(r'[^a-zA-Z\s]', '', str(text).lower())
    
    # Tokenize
    words = text.split()
    
    # Remove stop words and lemmatize
    processed_words = [
        lemmatizer.lemmatize(word) 
        for word in words 
        if word not in stop_words and len(word) > 2
    ]
    
    return ' '.join(processed_words)

# Apply preprocessing to Article column
df_pandas['Article'] = df_pandas['Article'].apply(preprocess_text)

# from sklearn.feature_extraction.text import CountVectorizer

# vectorizer = CountVectorizer()
# X = vectorizer.fit_transform(df_pandas['text'])
# feature_names = vectorizer.get_feature_names_out()

# # Convert sparse matrix to dense and create DataFrame
# X_dense = X.toarray()
# feature_df = pd.DataFrame(X_dense, columns=feature_names)

# # Combine with original metadata
# df_pandas = pd.concat([df_pandas[['article_id', 'date', 'publication']], feature_df], axis=1)

# print(f"Shape of count matrix: {X.shape}")
# print(f"Number of features: {len(feature_names)}")
# print(f"First 10 features: {feature_names[:10]}")

# df_pandas

In [9]:
import time
from sklearn.feature_extraction.text import CountVectorizer

# Time the CountVectorizer on Article column
start_time = time.time()

vectorizer = CountVectorizer(ngram_range=(2,3))
X = vectorizer.fit_transform(df_pandas['Article'])
feature_names = vectorizer.get_feature_names_out()

end_time = time.time()

print(f"CountVectorizer execution time: {end_time - start_time:.4f} seconds")
print(f"Shape of count matrix: {X.shape}")
print(f"Number of features: {len(feature_names)}")
print(f"First 10 features: {feature_names[:10]}")

# Convert sparse matrix to dense and create DataFrame
X_dense = X.toarray()
feature_df = pd.DataFrame(X_dense, columns=feature_names)

# Combine with original metadata
df_pandas_fin = pd.concat([df_pandas[['article_id', 'date']], feature_df], axis=1)

df_pandas_fin.head()

CountVectorizer execution time: 1.9710 seconds
Shape of count matrix: (2692, 685806)
Number of features: 685806
First 10 features: ['aabar investment' 'aabar investment biggest' 'aamer ali'
 'aamer ali gave' 'aamer ali khawar' 'aamer bismillah'
 'aamer bismillah khan' 'aamer ehsan' 'aamer ehsan zulfiqar' 'aamer nazir']


,article_id,date,aabar investment,aabar investment biggest,aamer ali,aamer ali gave,aamer ali khawar,aamer bismillah,aamer bismillah khan,aamer ehsan,...,zyl caught,zyl caught end,zyl dean,zyl dean elgar,zyl ended,zyl ended spliced,zyl run,zyl run four,zyl six,zyl six start
0,1,1/1/2015,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,1/2/2015,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,1/5/2015,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,1/6/2015,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1/6/2015,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
def create_ngrams_polars(df, text_col="text", n=2):
    """
    Memory-efficient n-gram creation using group_by instead of pivot
    """
    # Step 1: Create the base ngram counts (same as before)
    if n == 1:
        ngram_counts = (
            df.lazy()
            .with_columns(
                pl.col(text_col)
                .str.to_lowercase()
                .str.extract_all(r'(?u)\b\w\w+\b')
                .alias("words")
            )
            .explode("words")
            .group_by(["article_id", "date", "publication", "words"])
            .agg(pl.len().alias("count"))
            .rename({"words": "ngram"})
        ).collect()
    else:
        # Build n-gram logic (same as before)
        concat_elements = [pl.element()]
        filter_conditions = []
        
        for i in range(1, n):
            concat_elements.append(pl.element().shift(-i))
            filter_conditions.append(pl.element().shift(-i).is_not_null())
        
        combined_filter = filter_conditions[0]
        for condition in filter_conditions[1:]:
            combined_filter = combined_filter & condition
        
        ngram_counts = (
            df.lazy()
            .with_columns(
                pl.col(text_col)
                .str.to_lowercase()
                .str.extract_all(r'(?u)\b\w\w+\b')
                .alias("words")
            )
            .with_columns(
                pl.col("words").list.eval(
                    pl.concat_str(concat_elements, separator=" ")
                    .filter(combined_filter)
                ).list.drop_nulls().alias("ngram")
            )
            .select(["article_id", "date", "publication", "ngram"])
            .explode("ngram")
            .group_by(["article_id", "date", "publication", "ngram"])
            .agg(pl.len().alias("count"))
        ).collect(engine='gpu')
    
    # Step 2: Get vocabulary (for interpretability)
    vocabulary = (
        ngram_counts
        .select("ngram")
        .unique()
        # .sort("ngram")
        # .get_column("ngram")
    )
    fin_df = (
        ngram_counts.lazy()
        .group_by("article_id", "date", "publication")
        .agg(*[pl.col('count').filter(pl.col('ngram') == pl.lit(ngram)).sum().alias(f'{ngram}') for ngram in list(vocabulary.to_series())])
    ).collect(engine='gpu')
    
    return fin_df, vocabulary
    



In [30]:
ngram_cnt, vocab = create_ngrams_polars(pl.from_pandas(df_pandas), n=2, text_col='Article')

ModuleNotFoundError: GPU engine requested, but required package 'cudf_polars' not found.
Please install using the command `pip install cudf-polars-cu12` (or `pip install cudf-polars-cu11` if your system has a CUDA 11 driver).

In [24]:
ngram_cnt

article_id,date,publication,farmer welfare,based fixing,get cricket,rasool second,would cease,busy home,yasser corona,cpec envisions,weightlifter extended,history growing,ominous claycourt,manager afinitithe,turning legbreak,respect civil,done scale,sammys tenth,sand reservoir,hike sign,year got,intent lucky,day image,car rental,platinum gained,strongthe governor,flew oversebastian,led varying,giving memento,career slowed,dollar spur,dont contemplate,perennial hope,get wrong,research institute,system action,…,news priced,seventh seed,day playbadar,saying wanted,useful help,bale scorer,rider jorge,rose opening,situation led,breakthrough removing,account order,odis three,coming image,meldonium australian,decrease took,lot strength,calling people,head saab,around worldthis,called july,notehedge fund,lord thursday,gone unnoticed,china move,made mandatory,conference roland,improving law,back sanction,added want,cameo first,china tumbling,saidbr brsimbal,numero uno,looking service,unexpectedly smooth,following recent,innes senior
i64,str,str,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,…,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
1234,"""2/14/2016""","""Boston Globe""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
189,"""6/23/2015""","""Boston Globe""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
737,"""6/22/2016""","""Boston Globe""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
892,"""8/11/2016""","""Boston Globe""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2684,"""3/20/2017""","""Boston Globe""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
380,"""12/8/2015""","""Boston Globe""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1056,"""1/9/2016""","""Boston Globe""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1484,"""3/21/2016""","""Boston Globe""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
